# Evaluation в GigaLogger
В этом ноутбуке мы произведем оценку нашего RAG'а с помощью датасета и мощной LLM (gpt-4o)
И не только! Мы также замерим качество ответов на обычном GigaChat (без RAG), с обычным RAG, и Adaptive RAG.
У нас в боте используется Adaptive RAG.
Предыдущие шаги:
1. [Генерация синтетического датасета](1_generate_dataset.ipynb)
2. [Загрузка датасета в GigaLogger](2_gigalogger_create_dataset.ipynb)

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import getpass

def get_env_var(var_name):
    if var_name in os.environ:
        return os.environ[var_name]
    else:
        return getpass.getpass(f"Enter {var_name}: ")

import sys
sys.path.append("..")  # Add the parent folder to the sys.path

load_dotenv(find_dotenv())
os.environ["LANGFUSE_HOST"] = "https://gigalogger.demo.sberdevices.ru"
os.environ["LANGFUSE_PUBLIC_KEY"] = get_env_var("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = get_env_var("LANGFUSE_SECRET_KEY")

In [2]:
from langfuse import Langfuse
langfuse = Langfuse(timeout=10000)

## Цепочка для ответов

Определим промпт для цепочки, которая будет сравнивать наши ответы с правильными ответами

In [3]:
from langchain_core.prompts import PromptTemplate
COT_PROMPT = PromptTemplate(
    input_variables=["query", "context", "result"], template="""Ты учитель, оценивающий тест.
Тебе дан вопрос, контекст, к которому относится вопрос, и ответ студента. Тебе нужно оценить ответ студента как ПРАВИЛЬНЫЙ или НЕПРАВИЛЬНЫЙ, основываясь на контексте.
Опиши пошагово своё рассуждение, чтобы убедиться, что твой вывод правильный. Избегай просто указывать правильный ответ с самого начала.

Пример формата:
QUESTION: здесь вопрос
CONTEXT: здесь контекст, к которому относится вопрос
STUDENT ANSWER: здесь ответ студента
EXPLANATION: пошаговое рассуждение здесь
GRADE: CORRECT или INCORRECT здесь

Оценивай ответы студента ТОЛЬКО на основе их фактической точности. Игнорируй различия в пунктуации и формулировках между ответом студента и правильным ответом. Ответ студента может содержать больше информации, чем правильный ответ, если в нём нет противоречивых утверждений. Начнём!

QUESTION: "{query}"
CONTEXT: "{context}"
STUDENT ANSWER: "{result}"
EXPLANATION:"""
)

In [4]:
from langchain.evaluation import CotQAEvalChain
from langchain_openai import ChatOpenAI
from langchain_community.chat_models.gigachat import GigaChat

def cot_llm(query, output, expected_output):
    # Используйте мощную модель для лучшего сравнения ответов
    eval_llm = ChatOpenAI(temperature=0, model="gpt-4o")
    # eval_llm = GigaChat(model="GigaChat-Pro", profanity_check=False,)
    eval_chain = CotQAEvalChain.from_llm(llm=eval_llm, prompt=COT_PROMPT)
    resp = eval_chain.invoke({
        "query": query, "context": expected_output, "result": output
    })
    return eval_chain._prepare_output(resp)

Проверим работу цепочки оценки ответов

In [5]:
# Тут оценка неправильного ответа от LLM
cot_llm("Кто главный герой книги", "Кот", "Собака")

{'reasoning': 'EXPLANATION:\n1. Вопрос спрашивает о главном герое книги.\n2. Контекст указывает, что главным героем является "Собака".\n3. Ответ студента утверждает, что главным героем является "Кот".\n4. Ответ студента не совпадает с контекстом, который указывает на "Собаку" как главного героя.\n\nGRADE: INCORRECT',
 'value': 'INCORRECT',
 'score': 0}

In [6]:
# Тут оценка правильного ответа от LLM
cot_llm("Кто главный герой книги", "Кот", "Котик")

{'reasoning': 'EXPLANATION:\n1. Прочитаем вопрос: "Кто главный герой книги".\n2. Прочитаем контекст: "Котик".\n3. Прочитаем ответ студента: "Кот".\n4. Сравним ответ студента с контекстом. В контексте указано, что главный герой книги — "Котик". Ответ студента — "Кот".\n5. Рассмотрим, является ли "Кот" синонимом или сокращением от "Котик". В данном случае, "Кот" и "Котик" могут рассматриваться как одно и то же животное, просто в разных формах (уменьшительно-ласкательная форма "Котик" и обычная форма "Кот").\n6. Убедимся, что ответ студента не содержит противоречивых утверждений и соответствует контексту.\n\nGRADE: CORRECT',
 'value': 'CORRECT',
 'score': 1}

## Оценка
### Оценка ответов с обычным GigaChat

In [7]:
from langchain_community.chat_models import GigaChat
llm = GigaChat(model="GigaChat-Pro", profanity_check=False, timeout=6000)

In [12]:
import tqdm
dataset = langfuse.get_dataset("rag_dataset")

In [13]:
for item in tqdm.tqdm(dataset.items):
    handler = item.get_langchain_handler(run_name="llm_without_rag_12")
    try:
        generation = llm.invoke(input=item.input, config={"callbacks": [handler]}).content
        resp = cot_llm(item.input, generation, item.expected_output)
        handler.trace.score(
            name="cot_llm",
            value=resp['score'],
            comment=resp['reasoning']
        )
    except Exception as e:
        handler.trace.score(
            name="cot_llm",
            value=0,
            comment=str(e)
        )

100%|██████████| 35/35 [06:56<00:00, 11.89s/it]


Первый прогон сделан. Смотрим результат...
![скриншот прогона](media/llm_without_rag.png)
Результат вышел `0.23`.
Судя по всему GigaChat хорошо справляется с вопросами сам о себе, но про GigaChain отвечает слабо.
Теперь попробуем прогнать датасет с простым RAG
### Оценка ответов GigaChat + RAG(стандартный)

In [6]:
from graph import vector_store

In [13]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=vector_store.as_retriever())

In [14]:
qa_chain.invoke("Как обновить GigaChain?")

{'query': 'Как обновить GigaChain?',
 'result': 'Выполните команду bash pip install -U gigachain_community'}

In [15]:
import tqdm
dataset = langfuse.get_dataset("rag_dataset")

for item in tqdm.tqdm(dataset.items):
    handler = item.get_langchain_handler(run_name="llm_with_rag")
    try:
        generation = qa_chain.invoke(input=item.input, config={"callbacks": [handler]})['result']
        resp = cot_llm(item.input, generation, item.expected_output)
        handler.trace.score(
            name="cot_llm",
            value=resp['score'],
            comment=resp['reasoning']
        )
    except Exception as e:
        handler.trace.score(
            name="cot_llm",
            value=0,
            comment=str(e)
        )

100%|██████████| 35/35 [04:53<00:00,  8.38s/it]


Смотрим результат...
![скриншот прогона](media/llm_with_rag.png)
Результат вышел `0.51`.
### Оценка ответов GigaChat + Adaptive RAG

In [8]:
from graph import graph, GraphState

In [17]:
graph.invoke(input=GraphState(question="Как обновить GigaChain?"))['generation']

'Команда `bash pip install -U gigachain_community` поможет вам обновить GigaChain до последней версии. Если у вас возникнут вопросы или проблемы при установке, пожалуйста, обратитесь за дополнительной поддержкой.'

In [18]:
import tqdm
import logging
dataset = langfuse.get_dataset("rag_dataset")

for item in tqdm.tqdm(dataset.items):
    handler = item.get_langchain_handler(run_name="llm_with_arag")
    try:
        s = GraphState(question=item.input)
        generation = graph.invoke(input=s, config={"callbacks": [handler]})['generation']
        resp = cot_llm(item.input, generation, item.expected_output)
        handler.trace.score(
            name="cot_llm",
            value=resp['score'],
            comment=resp['reasoning']
        )
    except Exception as e:
        logging.error(e, exc_info=True)
        handler.trace.score(
            name="cot_llm",
            value=0,
            comment=str(e)
        )

 86%|████████▌ | 30/35 [08:13<01:00, 12.15s/it]Received 413 error by Langfuse server, not retrying: <html>
<head><title>413 Request Entity Too Large</title></head>
<body bgcolor="white">
<center><h1>413 Request Entity Too Large</h1></center>
<hr><center>nginx/1.14.1</center>
</body>
</html>

100%|██████████| 35/35 [09:38<00:00, 16.53s/it]


Смотрим результат...
![скриншот прогона](media/llm_with_arag.png)
Результат вышел `0.57`.

In [6]:
from graph_2 import graph, GraphState

/Users/19563044/Documents/giga/gigachain_telegram_bot/venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [10]:
import tqdm
import logging
dataset = langfuse.get_dataset("rag_dataset")

for item in tqdm.tqdm(dataset.items):
    handler = item.get_langchain_handler(run_name="llm_with_arag_18")
    try:
        s = GraphState(question=item.input)
        # generation = graph.invoke(input=s, config={"callbacks": [handler]})['generation']
        generation = graph.invoke(input=s)['generation']
        resp = cot_llm(item.input, generation, item.expected_output)
        handler.trace.score(
            name="cot_llm",
            value=resp['score'],
            comment=resp['reasoning']
        )
    except Exception as e:
        logging.error(e, exc_info=True)
        handler.trace.score(
            name="cot_llm",
            value=0,
            comment=str(e)
        )

100%|██████████| 35/35 [08:30<00:00, 14.59s/it]
